## Imports:

In [1]:
import scipy
import os
import pandas
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plt_lab
import matplotlib.gridspec as gridspec
import obspy
import pandas as pd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Load data:
We are going to use this data to plot spectograms of SCG signals. 

In [3]:
dir_path = "/Users/ecem/Documents/GitHub/seismocardiogram/data/diseased-dataset/Raw_Recordings"

In [4]:
ms_ = np.load(dir_path + "/ms.npy", allow_pickle= True)
mr_ = np.load(dir_path + "/mr.npy", allow_pickle= True)
as_ = np.load(dir_path + "/as.npy", allow_pickle= True)
ar_ = np.load(dir_path + "/ar.npy", allow_pickle= True)

In [5]:
ms_[0]

array([<obspy.core.trace.Trace object at 0x1042adc10>,
       <obspy.core.trace.Trace object at 0x12e671d00>], dtype=object)

In [6]:
ms_[0][0].stats

         network: 
         station: 
        location: 
         channel: 
       starttime: 1970-01-01T00:00:00.000000Z
         endtime: 1970-01-01T00:08:56.488281Z
   sampling_rate: 256.0
           delta: 0.00390625
            npts: 137342
           calib: 1.0
      processing: ["ObsPy 1.3.0: detrend(options={}::type='linear')", "ObsPy 1.3.0: taper(max_length=None::max_percentage=0.05::side='both'::type='hann')"]

## Trim 

In [7]:
npts = []
for i in range(len(ms_)):
    npts.append(ms_[i][0].stats.npts)

    
print(min(npts))
print(max(npts))

115688
146717


In [8]:
npts = []
for i in range(len(mr_)):
    npts.append(mr_[i][0].stats.npts)
    
print(min(npts))
print(max(npts))

107091
153064


In [9]:
npts = []
for i in range(len(as_)):
    npts.append(as_[i][0].stats.npts)
    
print(min(npts))
print(max(npts))

78527
307078


In [10]:
npts = []
for i in range(len(ar_)):
    npts.append(ar_[i][0].stats.npts)
    
print(min(npts))
print(max(npts))

121679
159196


In [11]:
for i in range(len(ms_)):
    ms_[i][0] = ms_[i][0].trim(ms_[i][0].stats.starttime , 
                          ms_[i][0].stats.starttime + 3000 * ms_[i][0].stats.delta)
    ms_[i][1] = ms_[i][1].trim(ms_[i][1].stats.starttime , 
                          ms_[i][1].stats.starttime + 3000 * ms_[i][0].stats.delta)
    ms_[i][2] = ms_[i][2].trim(ms_[i][2].stats.starttime , 
                          ms_[i][2].stats.starttime + 3000 * ms_[i][0].stats.delta)

In [12]:
for i in range(len(mr_)):
    mr_[i][0] = mr_[i][0].trim(mr_[i][0].stats.starttime , 
                          mr_[i][0].stats.starttime + 3000 * mr_[i][0].stats.delta)
    mr_[i][1] = mr_[i][1].trim(mr_[i][1].stats.starttime , 
                          mr_[i][1].stats.starttime + 3000 * mr_[i][0].stats.delta)
    mr_[i][2]  = mr_[i][2].trim(mr_[i][2].stats.starttime , 
                          mr_[i][2].stats.starttime + 3000 * mr_[i][0].stats.delta)
    

In [13]:
for i in range(len(as_)):
    as_[i][0] = as_[i][0].trim(as_[i][0].stats.starttime , 
                          as_[i][0].stats.starttime + 3000 * as_[i][0].stats.delta)
    as_[i][1] = as_[i][1].trim(as_[i][1].stats.starttime , 
                          as_[i][1].stats.starttime + 3000 * as_[i][0].stats.delta)
    as_[i][2] = as_[i][2].trim(as_[i][2].stats.starttime , 
                          as_[i][2].stats.starttime + 3000 * as_[i][0].stats.delta)

In [14]:
for i in range(len(ar_)):    
    ar_[i][0] = ar_[i][0].trim(as_[i][0].stats.starttime , 
                          as_[i][0].stats.starttime + 3000 * ar_[i][0].stats.delta)
    ar_[i][1] = ar_[i][1].trim(as_[i][1].stats.starttime , 
                          as_[i][1].stats.starttime + 3000 * ar_[i][0].stats.delta)
    ar_[i][2]  = ar_[i][2].trim(as_[i][2].stats.starttime , 
                          as_[i][2].stats.starttime + 3000 * ar_[i][0].stats.delta)

## comparison of two dataset:

Now we will to try visualize spectograms. So far we have used Obspy to visualize SCG signals. However, with obspy we are not able to plot spectograms side by side. This is important for us because we want to see spectograms of x,y and z components simultaneously for both healty and diseased people. So, from now on I will refer to those plots, for 3 components of healty and diseased (in total six plot, as "PAIR").


I have mentioned that Obpsy has a problem in side by side plotting. So I changed it's source code. 

Thats why from now on we are going to use **output_spectogram** function to get the coefficients needed to plot spectogram.

These are the imports needed for output_spectogram function to work.

In [15]:
from obspy.imaging.cm import obspy_sequential
import math
from matplotlib import mlab
from matplotlib.colors import Normalize

In [16]:
def _nearest_pow_2(x):
    a = math.pow(2, math.ceil(np.log2(x)))
    b = math.pow(2, math.floor(np.log2(x)))
    if abs(a - x) < abs(b - x):
        return a
    else:
        return b

In [17]:
def output_spectrogram(data, samp_rate, per_lap=0.9, wlen=None, log=False,
                outfile=None, fmt=None, axes=None, dbscale=False,
                mult=8.0, cmap=obspy_sequential, zorder=None, title=None,
                show=True, clip=[0.0, 1.0]):
 
    import matplotlib.pyplot as plt
    
    # enforce float for samp_rate
    samp_rate = float(samp_rate)

    # set wlen from samp_rate if not specified otherwise
    if not wlen:
        wlen = samp_rate / 100.

    npts = len(data)
    # nfft needs to be an integer, otherwise a deprecation will be raised
    # XXX add condition for too many windows => calculation takes for ever
    nfft = int(_nearest_pow_2(wlen * samp_rate))
    if nfft > npts:
        nfft = int(_nearest_pow_2(npts / 8.0))

    if mult is not None:
        mult = int(_nearest_pow_2(mult))
        mult = mult * nfft
    nlap = int(nfft * float(per_lap))

    data = data - data.mean()
    end = npts / samp_rate

    # Here we call not plt.specgram as this already produces a plot
    # matplotlib.mlab.specgram should be faster as it computes only the
    # arrays
    # XXX mlab.specgram uses fft, would be better and faster use rfft
    specgram, freq, time = mlab.specgram(data, Fs=samp_rate, NFFT=nfft,
                                         pad_to=mult, noverlap=nlap)
    # db scale and remove zero/offset for amplitude
    if dbscale:
        specgram = 10 * np.log10(specgram[1:, :])
    else:
        specgram = np.sqrt(specgram[1:, :])
    freq = freq[1:]
    
    vmin, vmax = clip
    if vmin < 0 or vmax > 1 or vmin >= vmax:
        msg = "Invalid parameters for clip option."
        raise ValueError(msg)
    _range = float(specgram.max() - specgram.min())
    vmin = specgram.min() + vmin * _range
    vmax = specgram.min() + vmax * _range
    norm = Normalize(vmin, vmax, clip=True)

    # calculate half bin width
    halfbin_time = (time[1] - time[0]) / 2.0
    halfbin_freq = (freq[1] - freq[0]) / 2.0

    # argument None is not allowed for kwargs on matplotlib python 3.3
    kwargs = {k: v for k, v in (('cmap', cmap), ('zorder', zorder))
              if v is not None}

    if log:
        # pcolor expects one bin more at the right end
        freq = np.concatenate((freq, [freq[-1] + 2 * halfbin_freq]))
        time = np.concatenate((time, [time[-1] + 2 * halfbin_time]))
        # center bin
        time -= halfbin_time
        freq -= halfbin_freq
        # Log scaling for frequency values (y-axis)
        ax.set_yscale('log')
        # Plot times
        ax.pcolormesh(time, freq, specgram, norm=norm, **kwargs)
    else:
        # this method is much much faster!
        specgram = np.flipud(specgram)
        # center bin
        extent = (time[0] - halfbin_time, time[-1] + halfbin_time,
                  freq[0] - halfbin_freq, freq[-1] + halfbin_freq)
    return [specgram, extent, end]

    



specgram : array-like


        2D array, columns are the periodograms of successive segments.

In [18]:
import scipy.signal

In [19]:
for i in range(len(ms_)):
    print( output_spectrogram(ms_[i][0].data, ms_[i][0].stats.sampling_rate, wlen=10)[0].shape)

(8192, 5)
(8192, 5)
(8192, 5)


In [20]:
for i in range(len(mr_)):
    print( output_spectrogram(mr_[i][0].data, mr_[i][0].stats.sampling_rate, wlen=10)[0].shape)

(8192, 5)
(8192, 5)
(8192, 5)
(8192, 5)
(8192, 5)
(8192, 5)
(8192, 5)
(8192, 5)
(8192, 5)
(8192, 5)
(8192, 5)
(8192, 5)
(8192, 5)


In [21]:
for i in range(len(as_)):
    print( output_spectrogram(as_[i][0].data, as_[i][0].stats.sampling_rate, wlen=10)[0].shape)

(8192, 5)
(8192, 5)
(8192, 5)
(8192, 5)
(8192, 5)
(8192, 5)
(8192, 5)
(8192, 5)
(8192, 5)
(8192, 5)
(1024, 106)
(1024, 106)
(1024, 106)
(1024, 106)
(1024, 106)
(1024, 106)


In [22]:
for i in range(len(ar_)):
    print( output_spectrogram(ar_[i][0].data, ar_[i][0].stats.sampling_rate, wlen=10)[0].shape)

(8192, 5)
(8192, 5)
(8192, 5)
(8192, 5)
(8192, 5)


In [23]:
from sklearn.manifold import TSNE

In [24]:
tsne = TSNE(random_state =42, perplexity = 30, init = "pca", learning_rate = 50, n_iter = 5000)

In [25]:
x = []
for i in range(len(ms_)):
    x.append(output_spectrogram(ms_[i][0].data, ms_[i][0].stats.sampling_rate, wlen=10)[0])
#x (100,128,50)

In [26]:
x = np.array(x)
x.shape

(3, 8192, 5)

In [27]:
t = []
for i in range(len(ms_)):
    t.append(tsne.fit_transform(x[i]))

KeyboardInterrupt: 

In [ ]:
t = np.array(t)
f = plt.figure(figsize=(8, 8))
for i in range(len(ms_)):
    ax = plt.subplot()
    sc = ax.scatter(t[i][:,0], t[i][:,1])

        
plt.show()